In [1]:
import sys
sys.path.append('../')

from util import load_node_csv, load_edge_csv
from model.light_gcn import LightGCN

from sklearn.model_selection import train_test_split
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.loader import DataLoader
from torch_sparse import SparseTensor

import torch
import random

In [2]:
class Args:
    pass

args = Args()
args.gpu = 'cuda:3'
args.data_path = "../data/kobaco.csv"
args.num_iters = 10000
args.batch_size = 512
args.lambda_val = 1e-6

In [3]:
user_mapping = load_node_csv(args.data_path, index_col='user_id')
item_mapping = load_node_csv(args.data_path, index_col='item_id')

num_users, num_items = len(user_mapping), len(item_mapping)

train_edge_index = torch.load('../data/train_edge_index.pt').type(torch.long)
test_edge_index = torch.load('../data/test_edge_index.pt').type(torch.long)

In [4]:
device = torch.device(args.gpu if torch.cuda.is_available() else 'cpu')
model = LightGCN(num_users, num_items).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_sparse_edge_index = SparseTensor(row=train_edge_index[0], col=train_edge_index[1], sparse_sizes=(num_users + num_items, num_users + num_items)).to(device)
test_sparse_edge_index = SparseTensor(row=test_edge_index[0], col=test_edge_index[1], sparse_sizes=(num_users + num_items, num_users + num_items)).to(device)

In [5]:
# function which random samples a mini-batch of positive and negative samples
def sample_mini_batch(batch_size, edge_index):
    """Randomly samples indices of a minibatch given an adjacency matrix

    Args:
        batch_size (int): minibatch size
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    return batch

# Binary Cross Entropy

In [6]:
# label = torch.cat([torch.ones([args.batch_size]), torch.zeros([args.batch_size])]).to(device)

# for n_iter in range(args.num_iters):
#     model.train()
#     batch = sample_mini_batch(args.batch_size, train_edge_index)
#     # batch_idx = torch.randperm(args.batch_size*2)
#     users, pos_items, neg_items = batch
#     users = torch.cat([users]*2).to(device)
#     items = torch.cat([pos_items, neg_items]).to(device)

#     users_emb_final, users_emb_init, items_emb_final, items_emb_init = model.forward(train_sparse_edge_index)

#     train_reg_loss = args.lambda_val * (users_emb_init.norm(2).pow(2) + items_emb_init.norm(2).pow(2))
#     logits = torch.mul(users_emb_final[users], items_emb_final[items])
#     logits = torch.sum(logits, dim=-1)
#     # train_loss = model.bce_loss(logits[batch_idx], label[batch_idx]) + train_reg_loss
#     train_loss = model.bce_loss(logits, label) + train_reg_loss
    
#     optimizer.zero_grad()
#     train_loss.backward()
#     optimizer.step()

#     if n_iter % 200 == 0 and n_iter > 0:
#         print(f"[Epoch {n_iter}/{args.num_iters}] train_loss: {round(train_loss.item(), 5)}")

# Bayesian Personalized Ranking

In [7]:
min_loss = 99999
SAVE_DIR = '../output'
MODEL_NAME = 'light-gcn'

for n_iter in range(args.num_iters):
    model.train()
    batch = sample_mini_batch(args.batch_size, train_edge_index)
    users, pos_items, neg_items = batch
    users = torch.cat([users]*2).to(device)
    items = torch.cat([pos_items, neg_items]).to(device)

    # forward propagation
    users_emb_final, users_emb_init, items_emb_final, items_emb_init = model.forward(train_sparse_edge_index)

    # mini batching
    users, pos_items, neg_items = batch
    users, pos_items, neg_items = users.to(device), pos_items.to(device), neg_items.to(device)
    users_emb_final, users_emb_init = users_emb_final[users], users_emb_init[users]
    pos_items_emb_final, pos_items_emb_init = items_emb_final[pos_items], items_emb_init[pos_items]
    neg_items_emb_final, neg_items_emb_init = items_emb_final[neg_items], items_emb_init[neg_items]

    train_loss = model.bpr_loss(users_emb_final, users_emb_init, pos_items_emb_final, pos_items_emb_init, neg_items_emb_final, neg_items_emb_init, args.lambda_val)
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    

    if n_iter % 100 == 0 and n_iter > 0:
        cur_loss = train_loss.item()
        print(f"[Epoch {n_iter}/{args.num_iters}] train_loss: {cur_loss:.4f} | min_loss : {min_loss:.4f}")
        if min_loss > cur_loss:
            min_loss = train_loss
            model.eval()
            model.to('cpu')
            torch.save(model.state_dict(), f'{SAVE_DIR}/model/{MODEL_NAME}.bin')
            model.train()
            model.to(device)

[Epoch 100/10000] train_loss: -0.6930 | min_loss : 99999.0000
[Epoch 200/10000] train_loss: -0.7045 | min_loss : -0.6930
[Epoch 300/10000] train_loss: -0.7819 | min_loss : -0.7045
[Epoch 400/10000] train_loss: -1.0355 | min_loss : -0.7819
[Epoch 500/10000] train_loss: -1.7168 | min_loss : -1.0355
[Epoch 600/10000] train_loss: -2.9225 | min_loss : -1.7168
[Epoch 700/10000] train_loss: -4.1676 | min_loss : -2.9225
[Epoch 800/10000] train_loss: -5.5043 | min_loss : -4.1676
[Epoch 900/10000] train_loss: -7.6257 | min_loss : -5.5043
[Epoch 1000/10000] train_loss: -10.0501 | min_loss : -7.6257
[Epoch 1100/10000] train_loss: -12.2618 | min_loss : -10.0501
[Epoch 1200/10000] train_loss: -12.6600 | min_loss : -12.2618
[Epoch 1300/10000] train_loss: -16.1201 | min_loss : -12.6600
[Epoch 1400/10000] train_loss: -18.7864 | min_loss : -16.1201
[Epoch 1500/10000] train_loss: -20.5749 | min_loss : -18.7864
[Epoch 1600/10000] train_loss: -24.0843 | min_loss : -20.5749
[Epoch 1700/10000] train_loss: -2